# Introduction

This tutorial introduces the basic Auto-PyTorch API together with the classes for featurized and image data.
So far, Auto-PyTorch covers classification and regression on featurized data as well as classification on image data.
For installing Auto-PyTorch, please refer to the github page.

**Disclaimer**: In this notebook, data will be downloaded from the openml project for featurized tasks and CIFAR10 will be downloaded for image classification. Hence, an internet connection is required.

# API

There are classes for featurized tasks (classification, multi-label classification, regression) and image tasks (classification). You can import them via:

In [4]:
from autoPyTorch import (AutoNetClassification,
                         AutoNetMultilabel,
                         AutoNetRegression,
                         AutoNetImageClassification,
                         AutoNetImageClassificationMultipleDatasets)

In [5]:
# Other imports for later usage
import pandas as pd
import numpy as np
import os as os
import openml
import json

Upon initialization of a class, you can specify its configuration. Later, you can override its configuration in each fit call. The *config_preset* allows to constrain the search space to one of *tiny_cs, medium_cs* or *full_cs*. These presets can be seen in *core/presets/*.

In [6]:
autonet = AutoNetClassification(config_preset="tiny_cs", result_logger_dir="logs/")

Here are some useful methods provided by the API:

In [7]:
# Get the current configuration as dict
current_configuration = autonet.get_current_autonet_config()

# Get the ConfigSpace object with all hyperparameters, conditions, default values and default ranges
hyperparameter_search_space = autonet.get_hyperparameter_search_space()

# Print all possible configuration options 
#autonet.print_help()

The most important methods for using Auto-PyTorch are ***fit***, ***refit***, ***score*** and ***predict***.

First, we get some data:

In [8]:
# Get data from the openml task "Supervised Classification on credit-g (https://www.openml.org/t/31)"
task = openml.tasks.get_task(task_id=31)
X, y = task.get_X_and_y()
ind_train, ind_test = task.get_train_test_split_indices()
X_train, Y_train = X[ind_train], y[ind_train]
X_test, Y_test = X[ind_test], y[ind_test]

***fit*** is used to search for a good configuration by fitting configurations chosen by the algorithm (by default BOHB). The incumbent configuration is then returned and stored in the class.

We recommend to have a look at the possible configuration options first. Some of the most important options allow you to set the budget type (epochs or time), run id and task id for cluster usage, tensorboard logging, seed and more.

Here we search for a configuration for 300 seconds with 60-100 s time for fitting each individual configuration.
Use the *validation_split* parameter to specify a split size. You can also pass your own validation set
via *X_val* and *Y_val*. Use *log_level="info"* or *log_level="debug"* for more detailed output.

In [9]:
autonet = AutoNetClassification(config_preset="tiny_cs", result_logger_dir="logs/")
# Fit (note that the settings are for demonstration, you might need larger budgets)
results_fit = autonet.fit(X_train=X_train,
                          Y_train=Y_train,
                          validation_split=0.3,
                          max_runtime=300,
                          min_budget=60,
                          max_budget=100,
                          refit=True)

# Save fit results as json
with open("logs/results_fit.json", "w") as file:
    json.dump(results_fit, file)

The Box-Cox transformation can only be applied to strictly positive data
Using yeo-johnson instead


***refit*** allows you to fit a configuration of your choice for a defined time. By default, the incumbent configuration is refitted during a *fit* call using the *max_budget*. However, *refit* might be useful if you want to fit on the full dataset or even another dataset or if you just want to fit a model without searching.

You can specify a hyperparameter configuration to fit (if you do not specify a configuration the incumbent configuration from the last fit call will be used):

In [10]:
# Create an autonet
autonet_config = {
    "result_logger_dir" : "logs/",
    "budget_type" : "epochs",
    "log_level" : "info", 
    "use_tensorboard_logger" : True,
    "validation_split" : 0.0
    }
autonet = AutoNetClassification(**autonet_config)

# Sample a random hyperparameter configuration as an example
hyperparameter_config = autonet.get_hyperparameter_search_space().sample_configuration().get_dictionary()

# Refit with sampled hyperparameter config for 120 s. This time on the full dataset.
results_refit = autonet.refit(X_train=X_train,
                              Y_train=Y_train,
                              X_valid=None,
                              Y_valid=None,
                              hyperparameter_config=hyperparameter_config,
                              autonet_config=autonet.get_current_autonet_config(),
                              budget=50)

# Save json
with open("logs/results_refit.json", "w") as file:
    json.dump(results_refit, file)

[INFO] [12:46:30:autonet] Start autonet with config:
{'embeddings': ['none'], 'lr_scheduler': ['cosine_annealing', 'plateau'], 'networks': ['shapedresnet'], 'over_sampling_methods': ['smote'], 'preprocessors': ['none', 'truncated_svd', 'power_transformer'], 'target_size_strategies': ['none', 'upsample', 'median'], 'result_logger_dir': 'logs/', 'budget_type': 'epochs', 'log_level': 'info', 'use_tensorboard_logger': True, 'validation_split': 0.0, 'hyperparameter_search_space_updates': None, 'categorical_features': None, 'dataset_name': None, 'run_id': '0', 'task_id': -1, 'algorithm': 'bohb', 'eta': 3, 'min_workers': 1, 'working_dir': '.', 'network_interface_name': 'eth0', 'memory_limit_mb': 1000000, 'run_worker_on_master_node': True, 'use_pynisher': True, 'refit_validation_split': 0.0, 'cross_validator': 'none', 'cross_validator_args': {}, 'min_budget_for_cv': 0, 'shuffle': True, 'imputation_strategies': ['mean', 'median', 'most_frequent'], 'normalization_strategies': ['none', 'minmax', 

The Box-Cox transformation can only be applied to strictly positive data
Using yeo-johnson instead


[INFO] [12:47:04:autonet] Finished train with budget 50.0: Preprocessing took 0s, Training took 33s, Wrap up took 0s. Total time consumption in s: 33
[INFO] [12:47:04:autonet] [AutoNet] Done with current split!
[INFO] [12:47:04:autonet] Aggregate the results across the splits
[INFO] [12:47:04:autonet] Process 1 additional result(s)
[INFO] [12:47:04:autonet] Done Refitting


***pred*** returns the predictions of the incumbent model. ***score*** can be used to evaluate the model on a test set. 

In [11]:
# See how the random configuration performs (often it just predicts 0)
score = autonet.score(X_test=X_test, Y_test=Y_test)
pred = autonet.predict(X=X_test)

print("Model prediction:", pred[0:10])
print("Accuracy score", score)

Model prediction: [[0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]
Accuracy score 80.0


Finally, you can also get the incumbent model as PyTorch Sequential model via

In [12]:
pytorch_model = autonet.get_pytorch_model()
print(pytorch_model)

Sequential(
  (0): Linear(in_features=20, out_features=276, bias=True)
  (1): Sequential(
    (0): ResBlock(
      (layers): Sequential(
        (0): BatchNorm1d(276, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Sigmoid()
        (2): Linear(in_features=276, out_features=276, bias=True)
        (3): BatchNorm1d(276, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): Sigmoid()
        (5): Dropout(p=0.36734979590227845, inplace=False)
        (6): Linear(in_features=276, out_features=276, bias=True)
      )
    )
    (1): ResBlock(
      (layers): Sequential(
        (0): BatchNorm1d(276, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Sigmoid()
        (2): Linear(in_features=276, out_features=276, bias=True)
        (3): BatchNorm1d(276, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): Sigmoid()
        (5): Dropout(p=0.36734979590227845, inplace=False)
        (6): Linea

# Featurized Data

All classes for featurized data (*AutoNetClassification*, *AutoNetMultilabel*, *AutoNetRegression*) can be used as in the example above. The only difference is the type of labels they accept.

# Image Data

Auto-PyTorch provides two classes for image data. *autonet_image_classification* can be used for classification for images. The *autonet_multi_image_classification* class allows to search for configurations for image classification across multiple datasets. This means Auto-PyTorch will try to choose a configuration that works well on all given datasets.

In [3]:
# Load classes
autonet_image_classification = AutoNetImageClassification(config_preset="full_cs", result_logger_dir="logs/")
autonet_multi_image_classification = AutoNetImageClassificationMultipleDatasets(config_preset="tiny_cs", result_logger_dir="logs/")

Could not find BOHB_Multi_KDE, replacing with object


For passing your image data, you have two options (note that arrays are expected):

I) Via a path to a comma-separated value file, which in turn contains the paths to the images and the image labels (note header is assumed to be None):

In [4]:
csv_dir = os.path.abspath("../../datasets/example.csv")

X_train = np.array([csv_dir])
Y_train = np.array([0])

II) directly passing the paths to the images and the labels

In [5]:
df = pd.read_csv(csv_dir, header=None)
X_train = df.values[:,0]
Y_train = df.values[:,1]

Make sure you specify *image_root_folders* if the paths to the images are not specified from your current working directory. You can also specify *images_shape* to up- or downscale images.

Using the flag *save_checkpoints=True* will save checkpoints to the result directory:

In [ ]:
autonet_image_classification.fit(X_train=X_train,
                                 Y_train=Y_train,
                                 images_shape=[3,32,32],
                                 min_budget=100,
                                 max_budget=300,
                                 max_runtime=600,
                                 save_checkpoints=True,
                                 images_root_folders=[os.path.abspath("../../datasets/example_images")],
                                 log_level="info")

Auto-PyTorch also supports some common datasets. By passing a comma-separated value file with just one line, e.g. "CIFAR10, 0" and specifying *default_dataset_download_dir* it will automatically download the data and use it for searching. Supported datasets are CIFAR10, CIFAR100, SVHN and MNIST.

In [6]:
path_to_cifar_csv = os.path.abspath("../../datasets/CIFAR10.csv")

autonet_image_classification.fit(X_train=np.array([path_to_cifar_csv]),
                                 Y_train=np.array([0]),
                                 min_budget=600,
                                 max_budget=900,
                                 max_runtime=1800,
                                 default_dataset_download_dir="./datasets",
                                 images_root_folders=["./datasets"],
                                 log_level="info")

[INFO] [11:32:27:autonet] Start autonet with config:
{'additional_logs': [],
 'additional_metrics': [],
 'algorithm': 'bohb',
 'batch_loss_computation_techniques': ['standard', 'mixup'],
 'budget_type': 'time',
 'cuda': True,
 'cv_splits': 1,
 'dataloader_cache_size_mb': 0,
 'dataloader_worker': 1,
 'dataset_order': None,
 'default_dataset_download_dir': './datasets',
 'eta': 3,
 'evaluate_on_train_data': True,
 'file_extensions': ['.png', '.jpg', '.JPEG', '.pgm'],
 'final_activation': 'softmax',
 'global_results_dir': None,
 'half_num_cv_splits_below_budget': 0,
 'hyperparameter_search_space_updates': None,
 'images_root_folders': ['./datasets'],
 'images_shape': [3, 32, 32],
 'increase_number_of_trained_datasets': False,
 'keep_only_incumbent_checkpoints': True,
 'log_level': 'info',
 'loss_modules': ['cross_entropy', 'cross_entropy_weighted'],
 'lr_scheduler': ['cosine_annealing',
                  'cyclic',
                  'step',
                  'adapt',
                  'pla

0 50000
Files already downloaded and verified
Files already downloaded and verified


[INFO] [11:47:45:autonet] Finished train with budget 900.0s, Training took 900s, Wrap up took 0s, Init took 0s, Train took 900s, Validation took 0s, Log functions took 0s, Cumulative time 900s.
Total time consumption in s: 900
[INFO] [11:47:46:autonet] Training ['resnet152'] with budget 900.0 resulted in score: -59.46610467158413 took 918.7896795272827 seconds
[INFO] [11:47:46:hpbandster.run_0.worker.mlgpu14.4500.-1] WORKER: registered result for job (0, 0, 0) with dispatcher
[INFO] [11:47:46:hpbandster] HBMASTER: Timelimit reached: wait for remaining 0 jobs
[INFO] [11:47:46:hpbandster] DISPATCHER: Dispatcher shutting down
[INFO] [11:47:46:hpbandster] DISPATCHER: shut down complete
[INFO] [11:47:47:autonet] Start autonet with config:
{'additional_logs': [],
 'additional_metrics': [],
 'algorithm': 'bohb',
 'batch_loss_computation_techniques': ['standard', 'mixup'],
 'budget_type': 'time',
 'cuda': True,
 'cv_splits': 1,
 'dataloader_cache_size_mb': 0,
 'dataloader_worker': 1,
 'dataset

0 50000
Files already downloaded and verified
Files already downloaded and verified


[INFO] [12:03:05:autonet] Finished train with budget 900.0s, Training took 900s, Wrap up took 0s, Init took 0s, Train took 900s, Validation took 0s, Log functions took 0s, Cumulative time 900s.
Total time consumption in s: 900


{'loss': -59.46610467158413,
 'optimized_hyperparameter_config': {'NetworkSelectorDatasetInfo:darts:auxiliary': False,
  'NetworkSelectorDatasetInfo:darts:drop_path_prob': 0.1,
  'NetworkSelectorDatasetInfo:darts:init_channels': 36,
  'NetworkSelectorDatasetInfo:darts:layers': 20,
  'CreateImageDataLoader:batch_size': 73,
  'ImageAugmentation:augment': False,
  'ImageAugmentation:cutout': False,
  'LossModuleSelectorIndices:loss_module': 'cross_entropy_weighted',
  'NetworkSelectorDatasetInfo:network': 'resnet152',
  'OptimizerSelector:optimizer': 'sgd',
  'SimpleLearningrateSchedulerSelector:lr_scheduler': 'adapt',
  'SimpleTrainNode:batch_loss_computation_technique': 'standard',
  'OptimizerSelector:sgd:learning_rate': 0.005042807661492666,
  'OptimizerSelector:sgd:momentum': 0.1748329156598709,
  'OptimizerSelector:sgd:weight_decay': 0.07558471538402955,
  'SimpleLearningrateSchedulerSelector:adapt:T_max': 563,
  'SimpleLearningrateSchedulerSelector:adapt:T_mult': 1.879929125554272,

For searching across multiple datasets, pass multiple csv files to the corresponding Auto-PyTorch class. Make sure your specify *images_root_folders* for each of them.

In [ ]:
autonet_multi_image_classification.fit(X_train=np.array([path_to_cifar_csv, csv_dir]),
                                       Y_train=np.array([0]),
                                       min_budget=1500,
                                       max_budget=2000,
                                       max_runtime=4000,
                                       default_dataset_download_dir="./datasets",
                                       images_root_folders=["./datasets", "./datasets/example_images"],
                                       log_level="info")